In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Type, Iterable

# import the neccesary pyquil modules
from qaoa.cost_function import QAOACostFunctionOnQVM, QAOACostFunctionOnWFSim
from pyquil.api import local_qvm, WavefunctionSimulator
from pyquil.paulis import PauliSum, PauliTerm

# import the QAOAParameters that we want to demo
from qaoa.parameters import AdiabaticTimestepsQAOAParameters,\
AlternatingOperatorsQAOAParameters, AbstractQAOAParameters, GeneralQAOAParameters,\
QAOAParameterIterator, FourierQAOAParameters

from vqe.optimizer import scipy_optimizer

In [2]:
hamiltonian = PauliSum.from_compact_str("0.7*Z0*Z1 + 1.2*Z0*Z2 + (-0.5)*Z0")
print("hamiltonian =", hamiltonian)

timesteps = 2
myAbstractParams = AbstractQAOAParameters([hamiltonian,timesteps])
print(myAbstractParams)

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2 + (-0.5+0j)*Z0
Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	single_qubit_coeffs: [-0.5]
	qubits_pairs: [[0, 1], [0, 2]]
	pair_qubit_coeffs: [0.7 1.2]
	timesteps: 2



In [3]:
# Specify some angles
betas          = [0.1, 0.6]
gammas_singles = [] ## NOTE THAT WHEN THIS IS EMPTY WE GET THE ERROR BELOW 
gammas_pairs   = [0.1, 0.3]
parameters = (betas, gammas_singles, gammas_pairs)

myAlternatingParams = AlternatingOperatorsQAOAParameters.from_AbstractParameters(myAbstractParams,parameters)
print(myAlternatingParams)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.1 0.6]
	gammas_singles: []
	gammas_pairs: [0.1 0.3]



In [4]:
# Set up the WavefunctionSimulator
sim = WavefunctionSimulator()

In [5]:
cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                        params=myAlternatingParams,
                                        sim=sim,
                                        return_standard_deviation=True,
                                        noisy=False,
                                        log=[])
res = scipy_optimizer(cost_function, myAlternatingParams.raw(), epsilon=1e-3,
                      maxiter=500)
res

IndexError: index 0 is out of bounds for axis 0 with size 0

Now let's pass in some zeros instead to `gamma_singles`, rather than leaving the list empty. We see it works now :)

In [6]:
# Specify some angles
betas          = [0.1, 0.6]
gammas_singles = [0, 0]
gammas_pairs   = [0.1, 0.3]
parameters = (betas, gammas_singles, gammas_pairs)

myAlternatingParams = AlternatingOperatorsQAOAParameters.from_AbstractParameters(myAbstractParams,parameters)

cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                        params=myAlternatingParams,
                                        sim=sim,
                                        return_standard_deviation=True,
                                        noisy=False,
                                        log=[])
res = scipy_optimizer(cost_function, myAlternatingParams.raw(), epsilon=1e-3,
                      maxiter=500)
res

     fun: -1.8000856391021018
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 477
  status: 1
 success: True
       x: array([ 1.03233144, -0.3046696 ,  0.16559006,  0.01804522, -0.51219558,
       -0.91665133])